# Working with TrackMate data

In [1]:
import pycellin as pc

# %matplotlib widget

## 0. Example data

In [2]:
# Main example.
trackmate_xml = "../sample_data/Ecoli_growth_on_agar_pad.xml"

# Other examples.
trackmate_xml_fusions = "../sample_data/Ecoli_growth_on_agar_pad_with_fusions.xml"

## 1. Creating a model from a TrackMate XML file

In [3]:
# Parse the XML file and create a Pycellin Model object
# that contains all the data from the XML file.
model = pc.load_TrackMate_XML(trackmate_xml)

In Pycellin, tracks from TrackMate are modeled by directed acyclic graphs and are called cell lineages. **It means that splitting events are allowed, they even are recommended if you want to take full advantage of Pycellin. However, MERGING EVENTS ARE NOT SUPPORTED.** If you try to use Pycellin on a lineage with merging events, it may crash or produce incorrect results, especially if you are computing features related to tracking.


In [4]:
model_fusions = pc.load_TrackMate_XML(trackmate_xml_fusions)

/mnt/data/Code/pycellin/pycellin/pycellin/io/trackmate/loader.py:1233: UserWarning: Unsupported data, 3 cell fusions detected. It is advised to deal with them before any other processing, especially for tracking related features. Crashes and incorrect results can occur. See documentation for more details.
  warnings.warn(fusion_warning)


`load_TrackMate_XML()` automatically check for fusion when loading a file, and will warn the user if any are found. It is then up to the user to decide what to do. **Be careful if you decide to proceed without first sorting out the fusions.**

Pycellin is creating one lineage per TrackMate track. \
The arguments `keep_all_spots` and `keep_all_tracks` allow you to keep (value to `True`) or discard (value to `False`) spots/tracks that were filtered out in TrackMate. These arguments are false by default.

We can display basic information about the model.

In [5]:
print(model)
print(f"This model contains {model.data.number_of_lineages()} lineages:")
for lin_ID, lineage in model.data.cell_data.items():
    print(f"- ID {lin_ID}: {lineage}")

Model named 'Ecoli_growth_on_agar_pad' with 3 lineages, built from TrackMate.
This model contains 3 lineages:
- ID 0: CellLineage of ID 0 with 152 cells and 151 links.
- ID 1: CellLineage of ID 1 with 189 cells and 188 links.
- ID 2: CellLineage of ID 2 with 185 cells and 184 links.


So here we have three different cell lineages that you can identify by their ID: 0, 1 and 2.

In [7]:
import networkx as nx

In [13]:
def plot_cell_trajectory(
    lineage: pc.CellLineage,
    feature: str,
    start_noi: int | None = None,
    end_noi: int | list[int] | None = None,
) -> None:
    if start_noi is None:
        start_noi = lineage.get_root()
        # TODO: deal with the case where there are multiple roots.

    if isinstance(end_noi, int):
        nodes = set(nx.shortest_path(lineage, start_noi, end_noi))
    elif isinstance(end_noi, list):
        nodes = set()
        for end in end_noi:
            nodes.update(nx.shortest_path(lineage, start_noi, end))
    # Sort by frame number for debugging.
    nodes = sorted(nodes, key=lambda x: lineage.nodes[x]["frame"])
    print(len(nodes), nodes)

    # Get a view on the subgraph created by the nodes.
    subgraph = lineage.subgraph(nodes)

In [6]:
lin0 = model.data.cell_data[0]
lin0.plot_tree()

In [18]:
plot_cell_trajectory(
    lin0,
    "AREA",
    # start_noi=9051,
    end_noi=9470,
)

12 [9051, 9102, 9090, 9136, 9167, 9274, 9195, 9218, 9294, 9335, 9481, 9470]


*lossless*

## 2. Accessing, computing and adding features

### 2.1 TrackMate features

### 2.2 Pycellin features

### 2.3 Custom features

## 3. Exporting the data

### 3.1 Back to TrackMate

### 3.2 In a CSV format